## Which TESS candidates should I target?
Use this notebook to filter a list of TESS Objects of Interst (TOI) for visibility at a specific site and time and for effective temperature. This template checks for visibility from the Calar Alto Observatory (CAHA) in Spain, but it is easy to adapt to a different use case.

#### Minimum Working Instructions:
- download a CSV file containing TESS alerts (from tev.mit.edu) and save it in `data/`
- Adjust the parameters (see below) to your needs
- execute the cells in this notebook 

In [1]:
import astroplan
import numpy as np
import datetime as dt
from astroquery.mast import Catalogs
from astroplan.plots import plot_airmass
from observability import *
from IPython.core.display import display, HTML


>>> from astroplan import download_IERS_A
>>> download_IERS_A()
 [astroplan.utils]


apply some convenient ipython magic

In [2]:
%load_ext autoreload
%autoreload 2

display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.max_columns', 500)
pd.set_option('precision',2)

### Specify Parameters
Define observational constraints, the Calar Alto Observatory, a maximum effective temperature.

In [5]:
alertsFile = 'data/toi-2019-04-15.csv'    # path to the alerts file
lastUpdate = '2019-04-10'                 # include only alerts added after this date
earliestObs = str(np.datetime64('today')) # earliest time we are interested in for observations
latestObs = '2019-12-30 12:00'            # latest time we are interested in for observations
minAltitude = 30                          # minimum elevation for visibility [deg]
Teff_max = 4000                           # maximum temperature for the M-dwarf-filter [K]
CAHA = defineCAHA()                       # define observing location

### Read alerts
Specify minimum time in column "Updated" (=last time we checked)

In [6]:
alerts, targets = targetsFromCSV(alertsFile, minUpdated=lastUpdate)
alerts

,src,tic_id,toi_id,Disposition,RA,Dec,Tmag,Tmag Err,Epoc,Epoc Err,Period,Period Err,Duration,Duration Err,Transit Depth,Transit Depth Err,Sectors,Created,Updated
0,spoc,408310006,576.01,PC,144.88,-20.98,8.85,0.02,1518.96,5.70e-04,5.44,2.29e-04,3.62,0.06,3034.68,51.81,8,2019-04-05 20:18:18.798868+00:00,2019-04-15 03:44:32.867322
1,spoc,296739893,620.01,PC,142.17,-12.17,10.10,0.05,1518.01,7.94e-04,5.10,3.58e-04,1.17,0.17,3393.72,167.37,8,2019-04-10 17:25:17.350276+00:00,2019-04-13 13:53:59.603313
2,spoc,30828562,621.01,PC,134.72,-43.45,8.11,0.02,1520.11,2.27e-04,3.11,5.20e-05,2.44,0.01,6343.20,58.35,8,2019-04-10 17:25:16.640255+00:00,2019-04-13 13:53:59.598658
3,spoc,83092282,622.01,PC,125.31,-46.48,8.54,0.02,1520.69,4.94e-04,6.40,2.72e-04,3.87,0.04,3892.87,49.67,8,2019-04-10 17:25:16.791866+00:00,2019-04-13 13:53:59.570680
4,spoc,101395259,623.01,PC,148.72,-23.33,10.18,0.02,1523.94,9.68e-04,15.15,1.36e-03,2.98,0.08,6230.44,152.06,8,2019-04-10 17:25:16.994832+00:00,2019-04-13 13:53:59.561199
5,qlp,287329267,595.01,PC,109.78,-59.30,9.95,0.02,1331.95,6.57e-03,7.64,4.20e-04,2.48,0.87,438.55,2.05,8 7 5 6 4 3 2 1,2019-04-01 16:51:56.391994+00:00,2019-04-13 13:50:35.668184
6,qlp,153651591,607.01,PC,121.01,-26.17,10.47,0.02,1492.56,4.02e-03,2.15,2.90e-04,1.81,0.20,1076.91,1.63,8 7,2019-04-01 16:51:57.948796+00:00,2019-04-13 13:43:35.886778
7,qlp,270841724,590.01,PC,132.07,-47.11,10.44,0.04,1522.02,1.03e-03,18.90,8.10e-04,9.07,0.17,24270.00,3.69,8,2019-04-01 16:52:00.866175+00:00,2019-04-13 13:43:13.888694
8,qlp,80275202,610.01,PC,121.53,-44.38,10.43,0.02,1495.34,3.48e-03,3.66,6.60e-04,2.76,0.82,2383.11,9.72,8,2019-04-01 16:51:58.329379+00:00,2019-04-13 13:42:56.753407
9,qlp,145982812,599.01,PC,133.15,-50.01,10.40,0.02,1518.47,3.20e-03,0.83,2.10e-04,1.75,0.05,4744.55,473.45,8,2019-04-01 16:52:03.087834+00:00,2019-04-13 13:42:39.845129


### Define Observing Constraints

In [7]:
constraints, start_time, end_time = define_constraints(minAltitude, earliestObs, latestObs)

### Check Observability from CAHA Given the Constraints

In [8]:
observables = check_observability(alerts, constraints, CAHA, targets, Time(earliestObs),
                                 Time(latestObs))
observables

,src,tic_id,toi_id,Disposition,RA,Dec,Tmag,Tmag Err,Epoc,Epoc Err,Period,Period Err,Duration,Duration Err,Transit Depth,Transit Depth Err,Sectors,Created,Updated
0,spoc,408310006,576.01,PC,144.88,-20.98,8.85,0.02,1518.96,5.70e-04,5.44,2.29e-04,3.62,0.06,3034.68,51.81,8,2019-04-05 20:18:18.798868+00:00,2019-04-15 03:44:32.867322
1,spoc,296739893,620.01,PC,142.17,-12.17,10.10,0.05,1518.01,7.94e-04,5.10,3.58e-04,1.17,0.17,3393.72,167.37,8,2019-04-10 17:25:17.350276+00:00,2019-04-13 13:53:59.603313
25,qlp,118021229,614.01,PC,128.48,-20.94,9.90,0.02,1518.28,2.74e-03,0.89,2.10e-04,1.62,0.20,820.28,1.76,8,2019-04-01 16:52:00.000516+00:00,2019-04-13 13:26:27.499957
27,qlp,77527511,613.01,PC,142.26,-5.38,9.89,0.02,1518.27,8.20e-04,0.78,8.00e-05,0.99,0.15,1317.87,1.49,8,2019-04-01 16:51:59.520221+00:00,2019-04-13 13:25:41.681329
32,qlp,262746281,603.01,PC,141.11,5.77,9.73,0.02,1523.79,2.57e-03,8.09,1.92e-03,7.86,0.92,2857.43,3.48,8 7,2019-04-01 16:52:03.994625+00:00,2019-04-13 05:26:32.701516
33,qlp,412961700,589.01,PC,127.22,-20.95,6.73,0.03,1520.16,2.34e-03,2.52,5.00e-04,1.47,0.23,465.43,1.13,8,2019-04-01 16:52:03.919765+00:00,2019-04-13 05:23:37.534533
34,spoc,405454160,563.01,PC,143.24,-17.34,10.38,0.02,1524.14,3.42e-03,NaN,NaN,3.77,0.26,2350.52,175.37,8,2019-04-05 20:18:18.746960+00:00,2019-04-12 18:28:38.615839
35,spoc,1003831,564.01,PC,130.30,-16.04,10.63,0.02,1518.20,5.54e-04,1.65,7.00e-05,0.76,0.18,3006.61,150.12,8,2019-04-05 20:18:16.258020+00:00,2019-04-11 17:53:43.210558
36,spoc,1103432,565.01,PC,130.58,-17.19,12.77,0.02,1519.87,1.24e-03,3.73,4.55e-04,3.87,0.11,17864.30,505.39,8,2019-04-05 20:18:16.314006+00:00,2019-04-11 17:53:43.205937
38,spoc,13349647,567.01,PC,131.58,-8.03,12.15,0.02,1518.51,3.56e-04,1.54,4.00e-05,1.87,0.03,19329.38,210.21,8,2019-04-05 20:18:16.423829+00:00,2019-04-11 17:53:43.196647


### Filter for M dwarf Spectral Type

Query TIC on MAST with observable objects, simultaneously filtering by effective temperature

In [9]:
observableMdwarfs = Catalogs.query_criteria(catalog="Tic",Teff=[0,Teff_max], ID=observables.tic_id).to_pandas()
observableMdwarfs

,ID,version,HIP,TYC,UCAC,TWOMASS,SDSS,ALLWISE,GAIA,APASS,KIC,objType,typeSrc,ra,dec,POSflag,pmRA,e_pmRA,pmDEC,e_pmDEC,PMflag,plx,e_plx,PARflag,gallong,gallat,eclong,eclat,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,TWOMflag,prox,w1mag,e_w1mag,w2mag,e_w2mag,w3mag,e_w3mag,w4mag,e_w4mag,GAIAmag,e_GAIAmag,Tmag,e_Tmag,TESSflag,SPFlag,Teff,e_Teff,logg,e_logg,MH,e_MH,rad,e_rad,mass,e_mass,rho,e_rho,lumclass,lum,e_lum,d,e_d,ebv,e_ebv,numcont,contratio,disposition,duplicate_id,priority,objID
0,296739893,20180610,NaN,NaN,390-050511,09284158-1209551,NaN,NaN,NaN,5982719,NaN,STAR,2mass,142.17,-12.17,2mass,38.01,2.00,-385.51,2.00,sblink,NaN,NaN,NaN,244.79,27.07,148.88,-25.58,13.69,0.04,12.26,2.40e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.84,0.03,8.20,0.05,7.95,0.03,AAA-111-111-000-0-0,11.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.10,0.05,cdwrf,cdwrf,3633.0,70.0,4.80,0.37,NaN,NaN,0.46,0.07,0.49,0.09,5.07,2.39,DWARF,0.03,1.04e-02,41.67,NaN,0.04,NaN,41.0,5.78e-02,NaN,NaN,6.25e-03,235557733
1,296780789,20180610,NaN,NaN,378-057908,09292334-1430407,NaN,J092923.30-143041.9,5689391929738059648,5983786,NaN,STAR,2mass,142.35,-14.51,tmgaia,-75.98,2.00,-80.86,2.00,sblink,NaN,NaN,NaN,246.92,25.70,149.98,-27.72,16.57,0.11,15.02,8.00e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.11,0.03,10.55,0.04,10.23,0.02,AAA-222-111-000-0-0,27.8,10.02,0.02,9.86,0.02,9.8,0.05,8.57,NaN,13.97,8.93e-04,12.52,0.04,cdwrf,cdwrf,3404.0,67.0,4.91,0.40,NaN,NaN,0.34,0.06,0.34,0.07,8.78,4.44,DWARF,0.01,4.66e-03,90.04,NaN,0.05,NaN,26.0,2.06e-02,NaN,NaN,2.10e-03,235881331
2,413248763,20180610,47103.0,6055-01283-1,342-056630,09360161-2139371,NaN,NaN,5664814198431308288,NaN,NaN,STAR,2mass,144.01,-21.66,tmgaia,138.72,0.09,-990.30,0.09,tgas,106.4,0.65,tgas,253.88,22.08,154.77,-33.77,12.36,0.15,10.91,1.00e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.34,0.03,6.74,0.03,6.47,0.02,AAA-111-111-000-0-0,49.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.73,3.46e-04,8.73,0.03,cdwrf,cdwrf,3505.0,90.0,4.91,0.06,NaN,NaN,0.36,0.01,0.38,0.01,8.17,0.75,DWARF,0.02,1.07e-03,19.55,0.06,0.06,NaN,38.0,3.45e-04,NaN,NaN,2.03e-02,237685728


--------------

### Output
Display the information we need in a convenient way and produce plots that facilitate target selection.

Merge the two tables

In [10]:
observableMdwarfs.rename(columns={'ID' : 'tic_id'}, inplace=True)
observableMdwarfs.tic_id = observableMdwarfs.tic_id.astype('int64')
observableMdwarfs = pd.merge(observables, observableMdwarfs, on='tic_id', how='inner')
candidateTargets = [t for t in targets if float(t.name) in observableMdwarfs.toi_id.unique()]
observableMdwarfs

,src,tic_id,toi_id,Disposition,RA,Dec,Tmag_x,Tmag Err,Epoc,Epoc Err,Period,Period Err,Duration,Duration Err,Transit Depth,Transit Depth Err,Sectors,Created,Updated,version,HIP,TYC,UCAC,TWOMASS,SDSS,ALLWISE,GAIA,APASS,KIC,objType,typeSrc,ra,dec,POSflag,pmRA,e_pmRA,pmDEC,e_pmDEC,PMflag,plx,e_plx,PARflag,gallong,gallat,eclong,eclat,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,TWOMflag,prox,w1mag,e_w1mag,w2mag,e_w2mag,w3mag,e_w3mag,w4mag,e_w4mag,GAIAmag,e_GAIAmag,Tmag_y,e_Tmag,TESSflag,SPFlag,Teff,e_Teff,logg,e_logg,MH,e_MH,rad,e_rad,mass,e_mass,rho,e_rho,lumclass,lum,e_lum,d,e_d,ebv,e_ebv,numcont,contratio,disposition,duplicate_id,priority,objID
0,spoc,296739893,620.01,PC,142.17,-12.17,10.10,0.05,1518.01,7.94e-04,5.10,3.58e-04,1.17,0.17,3393.72,167.37,8,2019-04-10 17:25:17.350276+00:00,2019-04-13 13:53:59.603313,20180610,NaN,NaN,390-050511,09284158-1209551,NaN,NaN,NaN,5982719,NaN,STAR,2mass,142.17,-12.17,2mass,38.01,2.00,-385.51,2.00,sblink,NaN,NaN,NaN,244.79,27.07,148.88,-25.58,13.69,0.04,12.26,2.40e-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.84,0.03,8.20,0.05,7.95,0.03,AAA-111-111-000-0-0,11.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.10,0.05,cdwrf,cdwrf,3633.0,70.0,4.80,0.37,NaN,NaN,0.46,0.07,0.49,0.09,5.07,2.39,DWARF,0.03,1.04e-02,41.67,NaN,0.04,NaN,41.0,5.78e-02,NaN,NaN,6.25e-03,235557733
1,spoc,296780789,573.01,PC,142.35,-14.51,12.52,0.04,1524.63,1.64e-04,13.58,2.32e-04,1.70,0.02,143013.76,3648.24,8,2019-04-05 20:18:17.905221+00:00,2019-04-11 17:53:43.082316,20180610,NaN,NaN,378-057908,09292334-1430407,NaN,J092923.30-143041.9,5689391929738059648,5983786,NaN,STAR,2mass,142.35,-14.51,tmgaia,-75.98,2.00,-80.86,2.00,sblink,NaN,NaN,NaN,246.92,25.70,149.98,-27.72,16.57,0.11,15.02,8.00e-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.11,0.03,10.55,0.04,10.23,0.02,AAA-222-111-000-0-0,27.8,10.02,0.02,9.86,0.02,9.8,0.05,8.57,NaN,13.97,8.93e-04,12.52,0.04,cdwrf,cdwrf,3404.0,67.0,4.91,0.40,NaN,NaN,0.34,0.06,0.34,0.07,8.78,4.44,DWARF,0.01,4.66e-03,90.04,NaN,0.05,NaN,26.0,2.06e-02,NaN,NaN,2.10e-03,235881331
2,spoc,413248763,562.01,PC,144.01,-21.66,8.73,0.03,1518.00,9.69e-04,3.93,2.66e-04,1.33,0.18,1164.26,66.07,8,2019-04-05 20:18:18.956776+00:00,2019-04-11 17:53:42.987237,20180610,47103.0,6055-01283-1,342-056630,09360161-2139371,NaN,NaN,5664814198431308288,NaN,NaN,STAR,2mass,144.01,-21.66,tmgaia,138.72,0.09,-990.30,0.09,tgas,106.4,0.65,tgas,253.88,22.08,154.77,-33.77,12.36,0.15,10.91,1.00e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.34,0.03,6.74,0.03,6.47,0.02,AAA-111-111-000-0-0,49.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.73,3.46e-04,8.73,0.03,cdwrf,cdwrf,3505.0,90.0,4.91,0.06,NaN,NaN,0.36,0.01,0.38,0.01,8.17,0.75,DWARF,0.02,1.07e-03,19.55,0.06,0.06,NaN,38.0,3.45e-04,NaN,NaN,2.03e-02,237685728


Display columns relevant for candidates spreadsheet

In [ ]:
to_spreadsheet = observableMdwarfs[['tic_id', 'toi_id', 'RA', 'Dec', 'Teff']]
to_spreadsheet

Print StarAlt-suited coordinate output for candidates

In [ ]:
coordList = [print('{} {} {}'.format(o.tic_id, o.ra, o.dec)) for i, o in observableMdwarfs.iterrows()]

check elevation at CAHA of the targets _tonight_

In [ ]:
fig, ax = plt.subplots()
ax = plot_airmass(candidateTargets, CAHA, str(dt.datetime.now().replace(hour=23, minute=59)), brightness_shading=True,
             altitude_yaxis=True, max_airmass=2.25)
ax.legend(facecolor='lightgray')
ax.grid(False, axis='y')
ax.axhspan(2, 3, alpha=.4, facecolor='w', edgecolor='maroon', hatch='//', linestyle='--')
plt.show()

visualize long-term (next few months) visibility of candidates

In [ ]:
plot_observability(candidateTargets, constraints, CAHA, Time(earliestObs), Time(latestObs))

------------------------------